In [43]:
import json
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import uuid
import numpy as np
from qdrant_client.http.models import QueryRequest, Vector

In [13]:
qd_client = QdrantClient("http://localhost:6333")

In [ ]:
embedding_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

#### Question 1

In [19]:
query = "I just discovered the course. Can I join now?"


query_embedding = next(embedding_model.embed(query))

In [20]:
print(query_embedding.min())

-0.11726373885183883


In [21]:

np.linalg.norm(query_embedding)

np.float64(1.0)

In [23]:
query_embedding.dot(query_embedding).round(2)

np.float64(1.0)

#### Question 2

In [24]:
query_doc = 'Can I still join the course after the start date?'

In [25]:
doc_embedding = next(embedding_model.embed(query_doc))

In [28]:
query_embedding.dot(doc_embedding)

np.float64(0.9008528895674548)

#### Question 3

In [29]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [34]:
collection_name = "homework_faq"

# Create collection
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=512,
        distance=models.Distance.COSINE,
    ),
)


True

In [ ]:

# Embed and upload documents
points = []
for idx, doc in enumerate(documents):
    embedding = next(embedding_model.embed([doc['text']]))
    point = models.PointStruct(
        id=idx,
        vector=embedding,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)


search_result = qd_client.search(
    collection_name=collection_name,
    query_vector=query_embedding,
    limit=1
)
# Print most similar document
top_match = search_result[0]
print("Most similar document index:", top_match.id)
print("Similarity score:", top_match.score)
print("Matched question:", top_match.payload["question"])


Most similar document index: 1
Similarity score: 0.8182378
Matched question: Course - Can I follow the course after it finishes?


/tmp/ipykernel_58163/514962876.py:19: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qd_client.search(


#### Question 4

In [47]:
points = []
for idx, doc in enumerate(documents):
    text = f"{doc['question']} {doc['text']}"
    embedding = next(embedding_model.embed(text))
    point = models.PointStruct(
        id=idx,
        vector=embedding,
        payload=doc
    )
    points.append(point)

qd_client.upsert(
    collection_name=collection_name,
    points=points
)



# Search with Qdrant
search_result = qd_client.search(
    collection_name=collection_name,
    query_vector=query_embedding,
    limit=1
)

# Print most similar document
top_match = search_result[0]
print("Most similar document index:", top_match.id)
print("Similarity score:", top_match.score)
print("Matched question:", top_match.payload["question"])


Most similar document index: 0
Similarity score: 0.8514543
Matched question: Course - Can I still join the course after the start date?


/tmp/ipykernel_58163/1776519811.py:20: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qd_client.search(
